CS 677 Final Project  
Sajin Shajee  
Sunny Gajjar  
Ahmed Arafaath  
Rohit Subramanian   

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Is this notebook running on Colab or Kaggle?
IS_COLAB = "google.colab" in sys.modules
IS_KAGGLE = "kaggle_secrets" in sys.modules

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0 is required
import tensorflow as tf
from typing import Dict, Text
from tensorflow import keras
from tqdm.notebook import tqdm
tqdm.pandas()
import tensorflow_recommenders as tfrs
try:
    if not tf.config.list_physical_devices('GPU'):
        assert tf.__version__ >= "2.0"
        print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
        if IS_COLAB:
            print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")
        if IS_KAGGLE:
            print("Go to Settings > Accelerator and select GPU.")
except:
    if not tf.test.is_gpu_available():
        assert tf.__version__ >= "2.0"
        print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
        if IS_COLAB:
            print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")
        if IS_KAGGLE:
            print("Go to Settings > Accelerator and select GPU.")

# Common imports
import numpy as np
import pandas as pd
import sklearn.preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import os
import datetime as dt
from pathlib import Path

# to make this notebook's output stable across runs
np.random.seed(42)
try:
    if not tf.config.list_physical_devices('GPU'):
        tf.random.set_seed(42)
    else:
        tf.random.set_random_seed(42)
except:
    if not tf.test.is_gpu_available():
        tf.random.set_seed(42)
    else:
        tf.random.set_random_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

No GPU was detected. LSTMs and CNNs can be very slow without a GPU.


zero_f function is suppose to convert customer id to string then adds to the right until the length of id is 10 characters long.

In [2]:
def zero_f(item):
    item=str(item)
    tem=len(item)
    if(len(item)<10):
        item=item.zfill(10)
    return item

In [3]:
trans_train = pd.read_csv('h-and-m-personalized-fashion-recommendations/transactions_train.csv',dtype={'customer_id': str,'article_id':str})
trans_train['quantity']=1
trans_train = trans_train[trans_train['t_dat'] >'2020-08-01']
articles= pd.read_csv('h-and-m-personalized-fashion-recommendations/articles.csv',dtype={'article_id': str,'product_code':str})
customers = pd.read_csv('h-and-m-personalized-fashion-recommendations/customers.csv',dtype={'customer_id':str})
master_df = trans_train[['customer_id','article_id','t_dat']].astype(str)
master_df['article_id']=master_df['article_id'].apply(zero_f)
master_df['quantity'] = trans_train['quantity'].astype(float)
masterdf = master_df
len(np.unique(trans_train['customer_id']))

359222

In [4]:
interactions = masterdf
interactions['t_dat']=pd.to_datetime(interactions['t_dat'])
train = interactions[interactions['t_dat']<='2020-09-15']
test = interactions[interactions['t_dat'] >'2019-09-15']
print(len(np.unique(train['customer_id'])))
train_ds = tf.data.Dataset.from_tensor_slices(dict(train[['customer_id','article_id']])).shuffle(100_000).batch(256).cache()
test_ds = tf.data.Dataset.from_tensor_slices(dict(test[['customer_id','article_id']])).batch(256).cache()

329034


In [5]:
items_dict = articles[['article_id']].drop_duplicates()
customer_dict=customers[['customer_id']].drop_duplicates()
items_dict = {name: np.array(value) for name, value in items_dict.items()}
customer_dict={name:np.array(value) for name,value in customer_dict.items()}
customers=tf.data.Dataset.from_tensor_slices(customer_dict)
items = tf.data.Dataset.from_tensor_slices(items_dict)
items = items.map(lambda x: x['article_id'])
customers=customers.map(lambda x: x['customer_id'])

In [6]:
### get unique item and user id's as a lookup table
unique_items = np.unique(np.concatenate(list(items.batch(1_000))))
unique_user_ids = np.unique(np.concatenate(list(customers.batch(1_000))))

# Randomly shuffle data and split between train and test.
tf.random.set_seed(42)

In [7]:
class CandidateModel(tfrs.Model):

    def __init__(self, user_model, item_model):
        super().__init__()
        
        self.item_model: tf.keras.Model = item_model
        self.user_model: tf.keras.Model = user_model
        
        ### for retrieval model. we take top-k accuracy as metrics
        metrics = tfrs.metrics.FactorizedTopK(candidates=items.batch(256).map(item_model))
        
        # define the task, which is retrieval                                    )    
        task = tfrs.tasks.Retrieval(metrics=metrics)
       
        self.task: tf.keras.layers.Layer = task

    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        # We pick out the user features and pass them into the user model.
        user_embeddings = self.user_model(features["customer_id"])
        # And pick out the movie features and pass them into the movie model,
        # getting embeddings back.
        positive_movie_embeddings = self.item_model(features["article_id"])

        # The task computes the loss and the metrics.
        return self.task(user_embeddings, positive_movie_embeddings)

In [ ]:
### Fitting and evaluating

### we choose the dimensionality of the query and candicate representation.
embedding_dimension = 64

## we pass the model, which is the same model we created in the query and candidate tower, into the model
item_model = tf.keras.Sequential([
                                tf.keras.layers.StringLookup(
                                vocabulary=unique_items, mask_token=None),
                                tf.keras.layers.Embedding(len(unique_items) + 1, embedding_dimension)
                                ])

user_model = tf.keras.Sequential([
                                tf.keras.layers.StringLookup(
                                vocabulary=unique_user_ids, mask_token=None),
                                # We add an additional embedding to account for unknown tokens.
                                tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension),
                                tf.keras.layers.Dense(embedding_dimension,activation="sigmoid"),
                                tf.keras.layers.Dense(embedding_dimension,activation="sigmoid")
                                ])

model = CandidateModel(user_model, item_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))
model.fit(train_ds,epochs=5,batch_size=128)
model.evaluate(test_ds, return_dict=True)

Epoch 1/5
1926/6849 [=======>......................] - ETA: 30:06 - factorized_top_k/top_1_categorical_accuracy: 1.6225e-05 - factorized_top_k/top_5_categorical_accuracy: 1.2777e-04 - factorized_top_k/top_10_categorical_accuracy: 3.2045e-04 - factorized_top_k/top_50_categorical_accuracy: 0.0013 - factorized_top_k/top_100_categorical_accuracy: 0.0019 - loss: 1419.5812 - regularization_loss: 0.0000e+00 - total_loss: 1419.5812

In [ ]:
unique_items

In [ ]:
unique_user_ids 

In [ ]:
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(items.batch(100).map(lambda items: (items,model.item_model(items))))

In [ ]:
j = '000231cc9af9e58ab4edc66fbd61da921b144ba85bc1c00d0ae2309531e4c210'
_, titles = index(tf.constant([j]),k=12)
print(f"Recommendations for user %s: {titles[0]}" %(j))

In [ ]:
def decoder(e):
    return e.decode('UTF-8')
def run_f(item):
    _, titles = index(tf.constant([item]),k=12)
    t = np.array(titles[0])
    vfunc = np.vectorize(decoder)
    l = vfunc(t)
    l = " ".join(l)
    return l

In [ ]:
submission_file = pd.read_csv('h-and-m-personalized-fashion-recommendations/sample_submission.csv',dtype={'customer_id': str})
sub_cust = submission_file["customer_id"]
sub_df = pd.DataFrame(columns=['Customer_Id', 'Article_Id'])
submission_file["prediction"] = submission_file['customer_id'].progress_apply(run_f)
submission_file

In [ ]:
submission_file.to_csv('submission.csv',index=False)